## About
It is anticipated that the `developer` exploring this blueprint will likely follow one of the two paths i.e.
* Build your own conversational agent from the grounds up
* Extend an existing agent

The blueprint has two sets of APIs the application builder is expected to interact with. The blueprint is expected to be deployed using docker compose for this APIs to be accesible.

* Agent apis
This is exposed on port 8081 and accessible on "http://IPADDR:8081".
Api documentation is available at  "http://IPADDR:8081/docs#"

* Analytics server apis
This is exposed on port 8082 and accessible on "http://IPADDR:8082".
Api documentation is available at  "http://IPADDR:8082/docs#" 

This notebook further illustrates one more aspect which becomes important when the `customer service operations` team wants to leverage the user feedback to power the data flywheel. Examples are included on how to glean the feedback data from the blueprint.

In [ ]:
!pip install requests

### Notebook variables

In [ ]:
IPADDRESS = "localhost" #Replace this with the correct IP address
AGENT_PORT = "8081"
ANALYTICS_PORT = "8082"
AGENT_BASE_URL = f'http://{IPADDRESS}:{AGENT_PORT}'
ANALYTICS_BASE_URL = f'http://{IPADDRESS}:{ANALYTICS_PORT}'

### Agent API usage
The next few set of cells illustrate examples of the APIs as documented at 
http://localhost:8081/docs#

In [ ]:
# Health
# Perform a Health Check
import requests
url = AGENT_BASE_URL + "/health"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())

In [ ]:
# Metrics

import requests
url = AGENT_BASE_URL + "/metrics"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
# create_session
# This needs to be done at the commencement of a conversation.
# The returned the session_id needs to be used in the conversation that ensues
import requests
url = AGENT_BASE_URL + "/create_session"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
if response.status_code == 200:
    try:
        data = response.json()
        session_id = data.get("session_id")
        print("Session ID:", session_id)
    except ValueError:
        print("Response is not in JSON format:", response.text)
else:
    print("Failed to create session. Status Code:", response.status_code)

In [ ]:
# generate
# user_id is set to John Doe (refer the customer data csv)
# session_id from the "create_session" is used in the post request

import requests
url = AGENT_BASE_URL + "/generate"  # Replace with the appropriate endpoint
headers = {
    "Content-Type": "application/json",
    "accept": "application/json"
}
payload = {
    "messages": [
        {
            "role": "user",
            "content": "What is the return status for my GeForce RTX 4070 SUPER??"
        }
    ],
    "user_id": "4165",  # Replace with the actual user ID
    "session_id": f"{session_id}"  # Replace with the actual session ID
}

response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
# feedback/response
# The feedback pertains to the most recent response as per "generate" api
# feedback convention: -1:Negative, 0:Neutral, 1=Positive
import requests

url = AGENT_BASE_URL + "/feedback/response"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "feedback": -1,
    "session_id": f"{session_id}"  # Replace with the actual session ID
}
response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
# end_session

import requests
url = f"{AGENT_BASE_URL}/end_session?session_id={session_id}"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

### Analytics API usage
The next few set of cells illustrate examples of the APIs as documented at 
http://localhost:8082/docs#

In [ ]:
# Health
# performs a health check
import requests
url = ANALYTICS_BASE_URL + "/health"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())

In [ ]:
# sessions
# Retrieve session information in last 2 hours

import requests

url = f"{ANALYTICS_BASE_URL}/sessions?hours=2"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
# conversation?session_id=xyz
# fetch the conversation history given a session id

import requests
print("session_id :{}".format(session_id))
url = f"{ANALYTICS_BASE_URL}/session/conversation?session_id={session_id}"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
# session/summary
# generate the conversation summary given a session_id

import requests
print("session_id :{}".format(session_id))
url = f"{ANALYTICS_BASE_URL}/session/summary?session_id={session_id}"
headers = {
    "accept": "application/json"
}
response = requests.get(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
## feedback/summary
## store the feedback for the summary generated by the solution
# feedback convention: -1:Negative, 0:Neutral, 1=Positive

url = ANALYTICS_BASE_URL + "/feedback/summary"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "feedback": +1, # positive
    "session_id": f"{session_id}"  # Replace with the actual session ID
}
response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
## feedback/session
## store user feedback for the overall conversation session.
# feedback convention: -1:Negative, 0:Neutral, 1=Positive

url = ANALYTICS_BASE_URL + "/feedback/session"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "feedback": -1, # negative
    "session_id": f"{session_id}"  # Replace with the actual session ID
}
response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
## feedback/sentiment
## store rating for the sentiment generated by the solution
# feedback convention: -1:Negative, 0:Neutral, 1=Positive

url = ANALYTICS_BASE_URL + "/feedback/sentiment"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "feedback": 0, # neutral
    "session_id": f"{session_id}"  # Replace with the actual session ID
}
response = requests.post(url, json=payload, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

In [ ]:
# delete_session

import requests
url = f"{AGENT_BASE_URL}/delete_session?session_id={session_id}"
headers = {
    "accept": "application/json"
}
response = requests.delete(url, headers=headers)
# Print the response
print("Status Code:", response.status_code)
try:
    print("Response Body:", response.json())
except ValueError:
    print("Response is not in JSON format:", response.text)

### Accessing User Feedback data
The next few set of cells illustrate how the various types of feedback data can be collected to power the data flywheel.
Refer to the docker-compose.yaml or helm chart for the credentials of the postgres db

In [7]:
POSTGRES_HOST = "localhost"
POSTGRESDB_PORT = "5432"
POSTGRES_USER = "postgres"
POSTGRES_PASSWD = "password"
POSTGRES_DBNAME = "postgres"
FEEDBACK_TBLNAME = "feedback"

#### Schema information of the `feedback` table

In [8]:
import psycopg2

# Connection details
host = POSTGRES_HOST
port = POSTGRESDB_PORT
database = POSTGRES_DBNAME
user = POSTGRES_USER
password = POSTGRES_PASSWD

# The schema and table you're interested in
schema_name = "public"      # replace if needed
table_name = FEEDBACK_TBLNAME   # replace with the actual table name

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cursor = conn.cursor()

    # Query to get column details of a specific table
    # information_schema.columns provides column_name and data_type
    query = """
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s AND table_schema = %s
        ORDER BY ordinal_position;
    """
    cursor.execute(query, (table_name, schema_name))

    columns = cursor.fetchall()

    # Print the schema details
    print(f"Schema for {schema_name}.{table_name}:")
    for col in columns:
        col_name, data_type = col
        print(f" - {col_name}: {data_type}")

except Exception as e:
    print("Error:", e)
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()

Schema for public.feedback:
 - session_id: character varying
 - sentiment: double precision
 - summary: double precision
 - session: double precision


#### Retrieve the feedback information for each session(session_id)
* sentiment
* summary
* session

These fields can take on a value such as

1:Positive,
0:Neutral,
-1:Negative

In [ ]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': POSTGRES_DBNAME,
    'user': POSTGRES_USER,
    'password': POSTGRES_PASSWD,
    'host': POSTGRES_HOST,      # e.g., 'localhost' or the IP address
    'port': POSTGRESDB_PORT   # e.g., '5432'
}

# Connect to the database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Query to select the first 5 rows from the customer_data table
query = f'SELECT session_id, sentiment, summary, session FROM feedback;'
# Execute the query
cur.execute(query)
rows = cur.fetchall()

# Print the headers and the corresponding rows
for i, row in enumerate(rows, start=1):
    print(f"{i}:{row}")

    # Close the connection
cur.close()
conn.close()

1:('string', -1.0, None, None)
2:('188b3c82-3384-4c6b-94cb-024d24b312d2', -1.0, 1.0, None)
